In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [3]:
class DenseCrossEntropy(nn.Module):
    def forward(self, x, target, weights=None):
        x = x.float()
        target = target.float()
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        loss = -logprobs * target
        loss = loss.sum(-1)
        return loss.mean()

In [25]:
df = pd.read_parquet("../data/train_orig_plus_pseudo_v1.parquet")
df.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language,split,source,hard_labels,soft_labels,logits
0,58210e39b3fd4441a2bd4a518bb44c2d,What is the difference between OpenCL and CUDA?,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...,model_b,chatglm-6b,koala-13b,English,train,lmsys_33k,1,"[0.015, 0.985]","[0.015, 0.985]"
1,90bfd142157948aba01931726c888e7f,"Fuji vs. Nikon, which is better?",Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...,model_b,koala-13b,oasst-pythia-12b,English,train,lmsys_33k,1,"[0.015, 0.985]","[0.015, 0.985]"
2,a7c5accc53e649a3bc6b2e41d962ebc4,How to build an arena for chatbots?,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...,model_b,vicuna-13b,oasst-pythia-12b,English,train,lmsys_33k,1,"[0.015, 0.985]","[0.015, 0.985]"
3,adf27e819a3c494cb6e993f0c660e097,When is it today?,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",model_a,vicuna-13b,koala-13b,English,train,lmsys_33k,0,"[0.985, 0.015]","[0.985, 0.015]"
4,c0fc42c6f5f14f2aa5a89f71f8553730,Count from 1 to 10 with step = 3,"1, 4, 7, 10\n\nCounting with a step of 3 means...","1, 4, 7, 10",model_a,vicuna-13b,koala-13b,English,train,lmsys_33k,0,"[0.985, 0.015]","[0.985, 0.015]"


In [31]:
loss_fn = DenseCrossEntropy()

logits = np.array(df["soft_labels"].tolist()[0:9])
logits = torch.tensor(logits)
logits.shape # (batch_size, num_classes)

torch.Size([9, 2])

In [32]:
predictions = np.array(df["soft_labels"].tolist()[1:10])
predictions = torch.tensor(predictions)
predictions.shape # (batch_size, num_classes)

torch.Size([9, 2])

In [36]:
loss_fn(logits, predictions)

tensor(0.5451)

In [34]:
ce_loss = nn.CrossEntropyLoss()
ce_loss(logits, predictions)

tensor(0.5451, dtype=torch.float64)

In [37]:
import torch.nn.functional as F
F.cross_entropy(logits, predictions)

tensor(0.5451, dtype=torch.float64)